# Prediction of Staining
This notebook can be used to predict images with existing models and plot the output, compare it to the target images, plot histograms etc.

In [ ]:
import os
import os.path
import numpy as np
import glob
import matplotlib.pyplot as plt
import preprocessing_functions as pre
import training_functions as training
import skimage.io
import prediction as pred
import keras

Load d0-images you want to convert and the d1 and d2 images for comaprison.

In [ ]:
# define directories
wrk_dir = r"C:\Users\tsche\Desktop\Master\2. Semester\Project Staining"
image_folder = 'Val_Images'
im_dir = os.path.join(wrk_dir, image_folder, '*.png')
imd0_dir = os.path.join(wrk_dir, image_folder, '*d0.png')

In [ ]:
#load images
d0_images = glob.glob(imd0_dir)
d0_stack = pre.stack_images(d0_images, num_channels = 1)
images = glob.glob(im_dir)
d12_images =  [x for x in images if x not in d0_images]
d12_stack= pre.stack_images(d12_images, num_channels = 2)

In [ ]:
# normalize images
(d0_norm, mean1, std1) = pre.norm_batch(d0_stack)
(d12_norm, mean, std) = pre.norm_batch(d12_stack)
#d0_norm = pre.recenter(d0_norm)
#d12_norm = pre.recenter(d12_norm)

Plot images and histograms if interested.

In [ ]:
# plot images
plt.figure(figsize=(10,50)) 
n = d0_stack.shape[0]
for p in range(n):
    plt.subplot(n,3,1+3*p)    
    plt.imshow(d0_stack[p], cmap='gray')
    plt.axis('off')
    if p == 0:
        plt.title('d0')

    plt.subplot(n,3,2+3*p)    
    plt.imshow(d12_stack[p,:,:,0], cmap='gray')
    plt.axis('off')
    if p == 0:
        plt.title('d1')
    
    plt.subplot(n,3,3+3*p)    
    plt.imshow(d12_stack[p,:,:,1], cmap='gray')
    plt.axis('off')
    if p == 0:
        plt.title('d2')

In [ ]:
# plot histograms
plt.figure(figsize=(15,10))
plt.subplot(2,3,1)
histogram, bin_edges = np.histogram(d0_stack[0], bins=65535, range=(0,65535))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("input")

plt.subplot(2,3,2)
histogram, bin_edges = np.histogram(d12_stack[0,:,:,0], bins=65535, range=(0,65535))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("d1 output")

plt.subplot(2,3,3)
histogram, bin_edges = np.histogram(d12_stack[0,:,:,1], bins=65535, range=(0,65535))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("d2 output")

plt.subplot(2,3,4)
histogram, bin_edges = np.histogram(d0_norm[0], bins=65535, range=(-1,10))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("Normalized input")

plt.subplot(2,3,5)
histogram, bin_edges = np.histogram(d12_norm[0,:,:,0], bins=65535, range=(-1,10))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("Normalized  d1 output")

plt.subplot(2,3,6)
histogram, bin_edges = np.histogram(d12_norm[0,:,:,1], bins=65535, range=(-1,10))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("Normalized  d2 output")

plt.show()

Load desired model and convert images

In [ ]:
model_name = 'd1d2_32im_ssim2'
mdl_dir = r'..\models'
model = keras.models.load_model(os.path.join(mdl_dir,model_name), compile = False)

In [ ]:
model_name = 'd1d2_32im'
mdl_dir = r'..\models'
model = keras.models.load_model(os.path.join(mdl_dir,model_name))

In [ ]:
# change input layer of model 
new_model = training.get_model(1104,1104)
new_model.set_weights(model.get_weights())

In [ ]:
output = new_model.predict(images, batch_size = 4, verbose = 1)
#output_norm = pre.center_back(output)
output_norm = pre.unnormalize(output_norm, mean, std)


In [ ]:
# save images 
num_im = output.shape[0]
im_dir = '..\images'
for i in range(num_im):
        im1 = output_norm[i,:,:,0]
        dir_d1 = os.path.join(im_dir, name + str(i) + '_d1.png')
        dir_d2 = os.path.join(im_dir, name + str(i) + '_d2.png')
        skimage.io.imsave(dir_d1, im1)
        #cv2.imwrite(dir_d1, im1.astype(np.uint16))
        im2 = output_norm[i,:,:,1]
        skimage.io.imsave(dir_d2, im2)
        #cv2.imwrite(dir_d2, im2.astype(np.uint16))

Plot predicted images and histograms of output.

In [ ]:
# plot predicted images
plt.figure(figsize=(10,50)) 
n = out.shape[0]
for p in range(n):
    plt.subplot(n,3,1+3*p)    
    plt.imshow(d0_stack[p], cmap='gray')
    if p == 0:
        plt.title('input image')

    plt.subplot(n,3,2+3*p)    
    plt.imshow(out[p,:,:,0], cmap='gray')
    if p == 0:
        plt.title('output network')
    
    plt.subplot(n,3,3+3*p)    
    plt.imshow(out[p,:,:,1], cmap='gray')
    if p == 0:
        plt.title('output network')

In [ ]:
# plot histogram of converted image
plt.figure(figsize=(15,10))
plt.subplot(2,3,1)
histogram, bin_edges = np.histogram(d12_stack[0,:,:,0], bins=65535, range=(0,65535))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("d1 target")

plt.subplot(2,3,4)
histogram, bin_edges = np.histogram(d12_stack[0,:,:,1], bins=65535, range=(0,65535))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("d2 target")

plt.subplot(2,3,2)
histogram, bin_edges = np.histogram(output_norm[0,:,:,0], bins=65535, range=(-1000,64535))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("Unnormalized d1 output")

plt.subplot(2,3,3)
histogram, bin_edges = np.histogram(output[0,:,:,0], bins=65535, range=(-1,10))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("d1 output")

plt.subplot(2,3,5)
histogram, bin_edges = np.histogram(output_norm[0,:,:,1], bins=65535, range=(-1000,64535))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("Unnormalized d2 output")

plt.subplot(2,3,6)
histogram, bin_edges = np.histogram(output[0,:,:,1], bins=65535, range=(-1,10))
plt.plot(bin_edges[0:-1], histogram)  
plt.title("d2 output")

#plt.savefig()
plt.show()